## text to speech

In [1]:
import pyttsx3

In [2]:
# text to speech
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voices',voices[0].id)

def speak(text):
    engine.say(text)
    print(text)
    engine.runAndWait()

In [3]:
speak("hello sukaran")

hello sukaran


## introduction

In [4]:
import datetime

def wish():
    hour = int(datetime.datetime.now().hour)

    if hour >= 6 and hour <= 12:
        speak("Good morning")
    elif hour > 12 and hour < 18:
        speak("Good afternoon")
    else:
        speak("Good evening")
    
    speak("I am your wikipedia assistant, what do you want to search")

In [5]:
wish()

Good morning
I am your wikipedia assistant, what do you want to search


## speech recognition

In [6]:
import speech_recognition as sr

In [7]:
def takecommand():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source,duration=1)
        print("listening...")
        r.pause_threshold = 1
        audio = r.listen(source)
    try:
        print("Recognizing...")
        q = r.recognize_google(audio,language='en-in')
        print(f"user said {q}")
    except UnboundLocalError:
        speak("Sorry could not recognise,try again")
        takecommand()
    except LookupError:                                 # speech is unintelligible
        print("Could not understand audio")
    finally:
        return q

### navigation starts

In [8]:
import time

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import wikipedia

In [10]:
# create window for navigation

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
# driver.maximize_window()
time.sleep(2)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.17/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Sukaran\.wdm\drivers\chromedriver\win32\95.0.4638.17]


## go to main wikipedia page

In [11]:
def search_page():
    driver.get("https://www.wikipedia.org")
    time.sleep(1)

## search something

In [12]:
def search_wikipedia(text):
    search_bar = driver.find_element_by_id("searchInput")
    search_bar.clear()
    time.sleep(1)
    search_bar.send_keys(text)
    search_bar.send_keys(Keys.RETURN)

## move up and down

In [13]:
def move_down():
    driver.execute_script("window.scrollTo(0, window.scrollY + 1000)")

In [14]:
def move_up():
    driver.execute_script("window.scrollTo(0, window.scrollY - 200)")

## parsing html 

In [15]:
def getAllContent():
    allHtmlContent = driver.page_source
    htmlContent = BeautifulSoup(allHtmlContent)
    reqHtmlContent = htmlContent.find(class_ = "mw-parser-output")
    all_links = list(reqHtmlContent.find_all('a'))
    all_headings = list(reqHtmlContent.find_all(['h2','h3']))
    all_paras = list(reqHtmlContent.find_all('p'))
    all_images = list(reqHtmlContent.find_all('img'))
    
    return all_links,all_headings,all_paras,all_images

## web page statistics

In [16]:
def webPageStats():
    page_title = driver.find_element_by_id("firstHeading").text.replace(' ','_')
    stats_url = f"https://pageviews.toolforge.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=0&range=latest-20&pages={page_title}"
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(stats_url)
    time.sleep(2)
    stats_list = driver.find_elements_by_class_name("linear-legend--counts")
    stats_string = ''
    for i in stats_list:
        if i.text != '':
            stats_string += i.text.rstrip() + "\n"
    #print(stats_string)
    speak(stats_string)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [17]:
search_page()

In [ ]:
while True:
    query = ''
    try:
        query = takecommand().lower()
    except UnboundLocalError:
        speak("Try again")
        
    if  "search page" or "main page" in query:
        search_page()
        time.sleep(1)
        
    if "wikipedia" in query:
        speak("searching wikipedia")
        query = query.replace("wikipedia","")
        search_wikipedia(query)
        
        # Scrapping content from wikipedia
        all_links,all_headings,all_paras,all_images = getAllContent()
        
        # initially
        curr_link = all_links[0]
        curr_heading = all_headings[1]
        curr_para = all_paras[0]
        curr_image = all_images[0]
        
        # length of all contents
        all_link_len = len(all_links)
        all_heading_len = len(all_headings)
        all_para_len = len(all_paras)
        all_image_len = len(all_images)
        
        
        
        # 2 lines , total link,heading,paras,images
        starting = f"This page has {all_link_len} links, {all_heading_len} headings, {all_para_len} paragraphs, {all_image_len} images"
        
        speak(starting)
        time.sleep(2)
        
        while True:

            # take speech input to be done.  ---------------
            speak("Tell me what to do")
            query2 = ''
            try:
                query2 = takecommand().lower()
                print("query2: " , query2)
            except UnboundLocalError:
                speak("Try again")
                

            #move up (working)
            if "up" in query2:
                # if already on the top, then speak feedback ---------------
                move_up()
                
            # move down
            elif "down" in query2:
                # if already on the bottom, then speak feedback ---------------
                move_down()
            
            # web page statistics  (working)
            elif "stats" in query2: 
                webPageStats()
                time.sleep(2)
                
            # next link
            elif "next link" in query2:
                curr_link_index = all_links.index(curr_link)
                if curr_link_index == all_link_len-1:
                    # do it for all others as well ---------------
                    speak("All links visited")
                elif curr_link_index < all_link_len:
                    curr_link_index += 1
                    curr_link = all_links[curr_link_index]
                
                # locating link in opened tab
                ActionChains(driver).move_to_element(driver.find_element_by_link_text(curr_link.get_text())).perform()
                time.sleep(1)
                speak(curr_link.get_text())

                
            # prev link
            elif "previous link" in query2:
                if curr_link_index == 0:
                    # do it for all others as well ---------------
                    speak("Already on first link")
                curr_link_index = all_links.index(curr_link)
                if curr_link_index > 0:
                    curr_link_index -= 1
                    curr_link = all_links[curr_link_index]
                    
                # locating link in opened tab                
                speak(curr_link.get_text())
                ActionChains(driver).move_to_element(driver.find_element_by_link_text(curr_link.get_text())).perform()
                time.sleep(1)
            
            
            #next heading ---------------
            elif "next heading" in query2:
                curr_heading_index = all_headings.index(curr_heading)
                if curr_heading_index < all_heading_len:
                    curr_heading_index += 1
                    curr_heading = all_headings[curr_heading_index]

                    
                #locating heading in tab
                speak(curr_heading.get_text())
                curr_heading_id = curr_heading.get_text().replace(" ","_").replace("[edit]",'')
                ActionChains(driver).move_to_element(driver.find_element_by_id(curr_heading_id)).perform()
                time.sleep(1)
           
        
        
            # prev heading ---------------
            elif "previous heading" in query2:
                curr_heading_index = all_headings.index(curr_heading)
                if curr_heading_index > 1:
                    curr_heading_index -= 1
                    curr_heading = all_headings[curr_heading_index]

                #locating heading in tab
                speak(curr_heading.get_text())
                curr_heading_id = curr_heading.get_text().replace(" ","_").replace("[edit]",'')
                ActionChains(driver).move_to_element(driver.find_element_by_id(curr_heading_id)).perform()
                time.sleep(1)
            
            # next para ---------------
            elif "next para" in query2 or "next paragraph" in query2:
                curr_para_index = all_paras.index(curr_para)
                if curr_para_index < all_para_len:
                    curr_para_index += 1
                    curr_para = all_paras[curr_para_index]
                
                speak(curr_para.get_text())
            
            
            # prev para ---------------
            elif "previous para" in query2 or "previous paragraph" in query2:
                curr_para_index = all_paras.index(curr_para)
                if curr_para_index > 0:
                    curr_para_index -= 1
                    curr_para = all_paras[curr_para_index]
              
                speak(curr_para.get_text())
            
            
            # next image/prev image  ---------------
            
            
            # go to main page  ---------------
            # then break this loop
            elif "main page" in query2 or "search page" in query2:
                speak("Main page loading...")
                search_page()
                break
            
            
            # go to google page   ("xyz google")  ---------------
            # first do what he says,then break this loop
            elif "google" in query2:
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[1])
                driver.get("https://www.google.com")
                time.sleep(2)
                search_bar = driver.find_element_by_name("q")
                search_bar.clear()
                search_text = query2.replace("google","")
                search_bar.send_keys(search_text)
                search_bar.send_keys(Keys.RETURN)
                time.sleep(4)
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            


listening...
Recognizing...
Try again
listening...
Recognizing...
user said Sachin Tendulkar Wikipedia
searching wikipedia


C:\Users\Sukaran\AppData\Local\Temp/ipykernel_8560/2419073478.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_id("searchInput")


This page has 3853 links, 47 headings, 108 paragraphs, 80 images
Tell me what to do
listening...
Recognizing...
user said next reading
query2:  next reading
Tell me what to do
listening...
Recognizing...
user said next heading
query2:  next heading
Early domestic career


C:\Users\Sukaran\AppData\Local\Temp/ipykernel_8560/2964749063.py:110: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ActionChains(driver).move_to_element(driver.find_element_by_id(curr_heading_id)).perform()


Tell me what to do
listening...
Recognizing...
user said next heading
query2:  next heading
Yorkshire
Tell me what to do
listening...
Recognizing...
user said next heading
query2:  next heading
International career
Tell me what to do
listening...
Recognizing...
user said previous heading
query2:  previous heading
Yorkshire


C:\Users\Sukaran\AppData\Local\Temp/ipykernel_8560/2964749063.py:125: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ActionChains(driver).move_to_element(driver.find_element_by_id(curr_heading_id)).perform()


Tell me what to do
listening...
Recognizing...
Try again
Tell me what to do
listening...
Recognizing...
user said previous heading
query2:  previous heading
Early domestic career
Tell me what to do
listening...
Recognizing...
user said next link
query2:  next link


C:\Users\Sukaran\AppData\Local\Temp/ipykernel_8560/2964749063.py:78: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ActionChains(driver).move_to_element(driver.find_element_by_link_text(curr_link.get_text())).perform()



Tell me what to do
listening...
Recognizing...
user said next link
query2:  next link
[1]
Tell me what to do
listening...
Recognizing...
user said next link
query2:  next link
Bombay
Tell me what to do
listening...
Recognizing...
user said next link
query2:  next link
Maharashtra
Tell me what to do
listening...
Recognizing...
user said next paragraph
query2:  next paragraph
Sachin Ramesh Tendulkar (/ˌsʌtʃɪn tɛnˈduːlkər/ (listen); pronounced [sət͡ʃin t̪eːɳɖulkəɾ]; born 24 April 1973) is a former international cricketer of India who served as captain of the Indian national team. He is widely regarded as one of the greatest batsmen in the history of cricket.[6] He is the highest run scorer of all time in international cricket, and the only player to have scored one hundred international centuries, the first batsman to score a double century in a One Day International (ODI), the holder of the record for the most runs in both Test and ODI cricket, and the only player to complete more than 